Multitask benchmark: https://www.nature.com/articles/s41597-019-0103-9

In [1]:
import yaml
with open('../../config.yaml') as f:
    config = yaml.full_load(f)

data_path = config['data_path']

In [2]:
import pandas as pd
from collections import defaultdict

In [3]:
df_train = pd.read_csv('train_listfile.csv')
df_val   = pd.read_csv('val_listfile.csv')
df_test  = pd.read_csv('test_listfile.csv')

In [4]:
df_train['y_true'].mean()

0.13534500374633882

## Removing non-metavision ICU stays

In [5]:
icustays = pd.read_csv('all_stays.csv')
icustays = icustays.sort_values(by=['SUBJECT_ID', 'INTIME', 'OUTTIME']).reset_index(drop=True)
metavision = icustays[icustays['DBSOURCE'] == 'metavision']['ICUSTAY_ID']

In [6]:
stays_by_subjects = defaultdict(list)
for i, (j, k) in icustays[['SUBJECT_ID', 'ICUSTAY_ID']].iterrows():
    stays_by_subjects[j].append(k)

In [8]:
my_labels = pd.read_csv('../' + data_path + 'population/mortality_48.0h.csv').set_index('ID')

In [9]:
df_out = []
for part, df_part in zip(['train', 'val', 'test'], [df_train, df_val, df_test]):
    for i, (name, y) in df_part.iterrows():
        try:
            ID, ep, _ = name.split('_')
            ID = int(ID)
            ep = int(ep[7:]) - 1
            stay_ID = stays_by_subjects[ID][ep]
            if stay_ID in metavision.values and stay_ID in my_labels.index:
                # Only keep patients that are recorded using metavision that have not died by 48 hour
                df_out.append((stay_ID, name, part, y, y))
                my_y = my_labels.loc[stay_ID, 'mortality_LABEL']
            else:
                continue
        except:
            print(name, ID, stay_ID, part, y, my_y)

In [10]:
df_out = pd.DataFrame(df_out, columns=['ID', 'stay', 'partition', 'mortality_LABEL', 'y_true'])

In [11]:
df_out = df_out.sort_values(by='ID')

In [12]:
len(df_out)

8577

In [13]:
df_out.to_csv('../' + data_path + 'population/pop.mortality_benchmark.csv', index=False)

In [14]:
df_out['mortality_LABEL'].mean()

0.12020519995336365